<a href="https://colab.research.google.com/github/NULabTMN/ps3-AnantShanbhag93/blob/master/ner_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, you will build a Viterbi decoder for an LSTM named-entity recognition model. As we mentioned in class, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. I cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

In this assignment, you will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, you will also implement some functions to evaluate the output of an NER system according to two metrics:
1. You will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. You will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [0]:
!pip install --upgrade spacy allennlp
import spacy
print(spacy.__version__)

Requirement already up-to-date: spacy in c:\users\shanb\anaconda3\envs\envpytorchcudapy36\lib\site-packages (2.2.4)
Requirement already up-to-date: allennlp in c:\users\shanb\anaconda3\envs\envpytorchcudapy36\lib\site-packages (0.9.0)


2.2.4


In [0]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003

torch.manual_seed(1)

In [0]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [0]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

vocab = Vocabulary.from_instances(train_dataset + validation_dataset)

562it [00:00, 12914.81it/s]
23it [00:00, 1160.85it/s]
100%|█████████████████████████████████████████████████████████████████████████████| 585/585 [00:00<00:00, 36679.94it/s]


## Define and Train Model

In [0]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()

accuracy: 0.8442, loss: 0.9104 ||: 100%|█████████████████████████████████████████████| 281/281 [00:04<00:00, 62.10it/s]
accuracy: 0.7878, loss: 1.1969 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 158.11it/s]
accuracy: 0.8442, loss: 0.7293 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 73.55it/s]
accuracy: 0.7878, loss: 1.1831 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 171.62it/s]
accuracy: 0.8442, loss: 0.7164 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 93.23it/s]
accuracy: 0.7878, loss: 1.1963 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 199.29it/s]
accuracy: 0.8442, loss: 0.7071 ||: 100%|█████████████████████████████████████████████| 281/281 [00:02<00:00, 94.15it/s]
accuracy: 0.7878, loss: 1.1840 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 166.63it/s]
accuracy: 0.8442, loss: 0.6992 ||: 100%|

accuracy: 0.8755, loss: 0.3387 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 93.65it/s]
accuracy: 0.8000, loss: 0.6632 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 180.79it/s]
accuracy: 0.8750, loss: 0.3360 ||: 100%|█████████████████████████████████████████████| 281/281 [00:02<00:00, 94.14it/s]
accuracy: 0.8082, loss: 0.6590 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 200.27it/s]
accuracy: 0.8777, loss: 0.3324 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 92.16it/s]
accuracy: 0.8143, loss: 0.6413 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 166.89it/s]
accuracy: 0.8778, loss: 0.3316 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 92.54it/s]
accuracy: 0.8061, loss: 0.6300 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 153.92it/s]
accuracy: 0.8780, loss: 0.3277 ||: 100%|

accuracy: 0.8987, loss: 0.2719 ||: 100%|█████████████████████████████████████████████| 281/281 [00:02<00:00, 94.48it/s]
accuracy: 0.8531, loss: 0.5193 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 200.27it/s]
accuracy: 0.8991, loss: 0.2685 ||: 100%|█████████████████████████████████████████████| 281/281 [00:02<00:00, 93.87it/s]
accuracy: 0.8449, loss: 0.5491 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 200.28it/s]
accuracy: 0.9004, loss: 0.2677 ||: 100%|█████████████████████████████████████████████| 281/281 [00:02<00:00, 95.74it/s]
accuracy: 0.8551, loss: 0.5184 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 200.27it/s]
accuracy: 0.9028, loss: 0.2644 ||: 100%|█████████████████████████████████████████████| 281/281 [00:02<00:00, 93.78it/s]
accuracy: 0.8592, loss: 0.5141 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 181.82it/s]
accuracy: 0.9027, loss: 0.2641 ||: 100%|

{'best_epoch': 98,
 'peak_cpu_memory_MB': 0,
 'peak_gpu_0_memory_MB': 590,
 'training_duration': '0:05:29.422421',
 'training_start_epoch': 0,
 'training_epochs': 99,
 'epoch': 99,
 'training_accuracy': 0.920453341807012,
 'training_loss': 0.21308933831672633,
 'training_cpu_memory_MB': 0.0,
 'training_gpu_0_memory_MB': 590,
 'validation_accuracy': 0.8591836734693877,
 'validation_loss': 0.47701976203825325,
 'best_validation_accuracy': 0.8714285714285714,
 'best_validation_loss': 0.42315542597013217}

## Evaluation

The simple code below creators a `predictor` object that applies the model to an input example and then loops over the examples in the validation set, printing out the input token, gold-standard output, and model output. You can see from these methods how to access data and model outputs for evaluation.

In [0]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)

def tag_sentence(s):
  tag_ids = np.argmax(predictor.predict_instance(s)['tag_logits'], axis=-1)
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

baseline_output = [tag_sentence(i) for i in validation_dataset]

Now, you can implement two evaluation functions: `violations` and `span_stats`.

In [0]:
# TODO: count the number of NER label violations,
# such as O followed by I-TYPE or B-TYPE followed by
# I-OTHER_TYPE
# Take tagger output as input
def violations(tagged):
  count = 0
  for each_sentence in tagged:
    prev_label = []
    for i in range(len(each_sentence)):
      each_triple = each_sentence[i]
      pred_label = each_triple[2].split("-")
#       print(pred_label)
      # check for begining of the sentence violation
      if i==0:
        if len(pred_label)>1 and pred_label[1]=="I":
#           print("violation")
          count +=1
      else:
        if pred_label[0]=="I":
          if prev_label[0]=="O":
#             print("violation")
            count +=1
          elif prev_label[0]=="B" and pred_label[1]!=prev_label[1]:
#             print("violation")
            count +=1
      prev_label = pred_label
#     print("\n")
  return count

# TODO: return the span-level precision, recall, and F1
# Take tagger output as input
def span_stats(tagged):
    list_all_spans_actual = []
    list_all_spans_pred = []
    
    for j in range(len(tagged)):
        each_sentence = tagged[j]
        start_actual_label = []
        contains_actual = []
        start_pred_label = []
        contains_pred = []
        for i in range(len(each_sentence)):
            each_triple = each_sentence[i]
            actual_label = each_triple[1].split("-")
            pred_label = each_triple[2].split("-")
            
            if actual_label[0]=='B' and len(contains_actual)==0:
                start_actual_label = actual_label
                contains_actual.append(i)
            elif actual_label[0]=='I':
                contains_actual.append(i)
            elif actual_label[0]=='O' and len(contains_actual)>0:
                list_all_spans_actual.append((start_actual_label[1], contains_actual[0], contains_actual[-1], j))
                contains_actual = []
                start_actual_label = []
            elif actual_label[0]=='B' and len(contains_actual)>0:
                list_all_spans_actual.append((start_actual_label[1], contains_actual[0], contains_actual[-1], j))
                contains_actual = []
                contains_actual.append(i)
                start_actual_label = start_actual_label
            
            
            if pred_label[0]=='B' and len(contains_pred)==0:
                start_pred_label = pred_label
                contains_pred.append(i)
            elif pred_label[0]=='I' and len(start_pred_label)>1 and start_pred_label[1]==pred_label[1]:
                contains_pred.append(i)
            elif pred_label[0]=='I' and len(start_pred_label)>1 and start_pred_label[1]!=pred_label[1]:
                contains_pred = []
                start_pred_label = []
            elif pred_label[0]=='O' and len(contains_pred)>0:
                list_all_spans_pred.append((start_pred_label[1], contains_pred[0], contains_pred[-1], j))
                contains_pred = []
                start_pred_label = []
            
    count_correct_matches = 0
    for each_tuple in list_all_spans_pred:
        if each_tuple in list_all_spans_actual:
            count_correct_matches += 1
    precision = count_correct_matches/len(list_all_spans_pred)
    recall = count_correct_matches/len(list_all_spans_actual)
    f1 = (2* precision* recall)/(precision + recall)   
    return {'precision': precision,
          'recall': recall,
          'f1': f1}

## You can check how many violations are made by the model output in predictor.
violations(baseline_output)

31

In [0]:
span_stats(baseline_output)

{'precision': 0.7, 'recall': 0.16279069767441862, 'f1': 0.2641509433962264}

## Decoding

Now you can finally implement the simple Viterbi decoder. The `predictor` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the line `predictor.predict_instance(i)['tag_logits']` in the code above.

Then, you will construct a transition matrix. You can use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the predictor object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run your `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [0]:
# This code show how to map from output vector components to labels
print(vocab.get_index_to_token_vocabulary('labels'))




vocab_dict = vocab.get_index_to_token_vocabulary('labels')
vocab_size = len(vocab_dict)
transition = np.zeros((vocab_size, vocab_size), dtype=float)
transition.fill('-inf')
for indexi,labeli in vocab_dict.items():
    row_label = labeli.split("-")
    for indexj,labelj in vocab_dict.items():
        col_label = labelj.split("-")
        if row_label[0]=='O' and col_label[0]!='I':
            transition[indexi][indexj] = 0 
        elif (row_label[0]=='B' or row_label[0]=='I') and col_label[0]=='O':
            transition[indexi][indexj] = 0
        elif row_label[0]=='B' and col_label[0]=='I' and row_label[1]==col_label[1]:
            transition[indexi][indexj] = 0 
        elif (row_label[0]=='B' or row_label[0]=='I') and col_label[0]=='B':
            transition[indexi][indexj] = 0
        elif row_label[0]=='I' and col_label[0]=='I' and row_label[1]==col_label[1]:
            transition[indexi][indexj] = 0 

            
            
            
pi = []
for indexi,labeli in vocab_dict.items():
    row_label = labeli.split("-")
    if row_label[0]=='O' or row_label[0]=='B':
        pi.append(0)
    else:
        pi.append('-inf')
pi = np.array(pi, dtype=float)



            
def get_emission_prob(s):
    return predictor.predict_instance(s)['tag_logits']


{0: 'O', 1: 'B-GPE', 2: 'I-ORG', 3: 'I-DATE', 4: 'B-CARDINAL', 5: 'I-EVENT', 6: 'B-PERSON', 7: 'B-NORP', 8: 'B-DATE', 9: 'B-ORG', 10: 'B-LOC', 11: 'I-LOC', 12: 'I-FAC', 13: 'I-PERSON', 14: 'I-GPE', 15: 'I-CARDINAL', 16: 'B-EVENT', 17: 'I-TIME', 18: 'I-WORK_OF_ART', 19: 'B-ORDINAL', 20: 'B-FAC', 21: 'B-TIME', 22: 'I-LAW', 23: 'I-QUANTITY', 24: 'I-NORP', 25: 'I-MONEY', 26: 'B-MONEY', 27: 'B-WORK_OF_ART', 28: 'B-QUANTITY', 29: 'B-LAW', 30: 'B-PRODUCT', 31: 'I-PRODUCT', 32: 'B-PERCENT', 33: 'I-PERCENT'}


In [0]:
# viterbi algorithm

def viterbi_decoder(sentence):
    emission_prob = np.array(get_emission_prob(sentence))
    T = len(emission_prob)
    N = len(emission_prob[0])
    viterbi_dp = np.zeros((N, T))
    path_prob_matrix = np.zeros((N, T), dtype=int)
    path_prob_matrix.fill(-1)
    viterbi_dp[:,0] = np.add(pi, emission_prob[0])
    
    for t in range(1,T):
        for s in range(N):
            product = np.add(np.add(viterbi_dp[:, t-1], transition[:, s]), emission_prob[t, s])
            index = np.argmax(product)
            viterbi_dp[s, t] = product[index]
            path_prob_matrix[s, t] = index
            
    ind = np.argmax(viterbi_dp[:,t])
    
    tag_ids = []
    while ind != -1:
        tag_ids.append(ind)
        ind = path_prob_matrix[ind,t]
        t -= 1
        
    tag_ids.reverse()
    fields = zip(sentence['tokens'], sentence['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
    return list(fields)

    
viterbi_output = [viterbi_decoder(i) for i in validation_dataset]

In [0]:
violations(viterbi_output)

0

In [0]:
span_stats(viterbi_output)

{'precision': 0.32432432432432434, 'recall': 0.27906976744186046, 'f1': 0.3}